In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim.downloader as api

# Download the Word2Vec model
word2vec_model = api.load('word2vec-google-news-300')


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

# Download NLTK data
nltk.download('punkt')

# Load the AG News dataset
dataset = load_dataset("ag_news")

# NLTK Tokenizer Function
def nltk_tokenizer(text):
    return word_tokenize(text.lower())

# Convert to PyTorch tensors
class AGNewsDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_length):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized_text = [self.vocab.get(token, self.vocab['<UNK>']) for token in text]
        if len(tokenized_text) < self.max_length:
            tokenized_text += [self.vocab['<PAD>']] * (self.max_length - len(tokenized_text))
        else:
            tokenized_text = tokenized_text[:self.max_length]
        return torch.tensor(tokenized_text, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Build vocabulary
def build_vocab(dataset, tokenizer):
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for example in dataset:
        tokens = tokenizer(example['text'])
        for token in tokens:
            if token not in vocab:
                vocab[token] = len(vocab)
    return vocab

# Tokenize and build vocab
train_texts = [nltk_tokenizer(example['text']) for example in dataset['train']]
train_labels = [example['label'] for example in dataset['train']]
vocab = build_vocab(dataset['train'], nltk_tokenizer)

# Set max length for padding
max_length = 128

# Create dataset
full_dataset = AGNewsDataset(train_texts, train_labels, vocab, max_length)

# Split training set into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Prepare test dataset
test_texts = [nltk_tokenizer(example['text']) for example in dataset['test']]
test_labels = [example['label'] for example in dataset['test']]
test_dataset = AGNewsDataset(test_texts, test_labels, vocab, max_length)


[nltk_data] Downloading package punkt to /home/IAIS/rrao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Initialize embedding matrix
def build_embedding_matrix(vocab, word2vec_model, embedding_dim):
    embedding_matrix = np.zeros((len(vocab), embedding_dim))
    for word, idx in vocab.items():
        if word in word2vec_model:
            embedding_matrix[idx] = word2vec_model[word]
        else:
            embedding_matrix[idx] = np.random.normal(size=(embedding_dim,))
    return torch.tensor(embedding_matrix, dtype=torch.float32)

# Build the embedding matrix
embedding_dim = 300  # Word2Vec uses 300-dimensional vectors
embedding_matrix = build_embedding_matrix(vocab, word2vec_model, embedding_dim)


In [5]:
class DeepLSTMModel(nn.Module):
    def __init__(self, vocab_size, output_dim, embedding_matrix):
        super(DeepLSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), 256, num_layers=3, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(256, output_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, cell) = self.lstm(x)
        return self.fc(hidden[-1])


In [9]:
from tqdm import tqdm

In [10]:
# Training Parameters
BATCH_SIZE = 64
EPOCHS = 10
OUTPUT_DIM = 4
LR = 0.001

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model, loss function, and optimizer
model = DeepLSTMModel(len(vocab), OUTPUT_DIM, embedding_matrix).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for texts, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch'):
        texts, labels = texts.to(device), labels.to(device)
        
        outputs = model(texts)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for texts, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch'):
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    
    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Validation Accuracy: {val_accuracy:.4f}')

# Final evaluation on test set
model.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for texts, labels in tqdm(test_loader, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch'):
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

overall_accuracy = accuracy_score(test_labels, test_preds)
class_report = classification_report(test_labels, test_preds, target_names=['World', 'Sports', 'Business', 'Sci/Tech'])

print(f'Test Accuracy: {overall_accuracy:.4f}')
print('Classification Report:')
print(class_report)


Epoch 1/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:06<00:00, 61.71batch/s]


Epoch 1/10, Validation Accuracy: 0.3658


Epoch 2/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:06<00:00, 57.44batch/s]


Epoch 2/10, Validation Accuracy: 0.6861


Epoch 3/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:06<00:00, 56.86batch/s]


...

Epoch 9/10, Validation Accuracy: 0.9136


Epoch 10/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:06<00:00, 57.26batch/s]


Epoch 10/10, Validation Accuracy: 0.9115


Epoch 10/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:02<00:00, 58.76batch/s]


Test Accuracy: 0.9147
Classification Report:
              precision    recall  f1-score   support

       World       0.91      0.93      0.92      1900
      Sports       0.97      0.96      0.97      1900
    Business       0.88      0.88      0.88      1900
    Sci/Tech       0.89      0.88      0.89      1900

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600

